# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 4: уменьшение размеров модели
### Оценивание и штрафы

Максимально допустимая оценка за работу — __14 баллов__. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн 20.12.23__ \
__Жесткий дедлайн 20.12.23__

### О задании

В этом задании вам предстоит научиться решать задачу Named Entity Recognition (NER) на самом популярном датасете – [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003). В вашем распоряжении будет предобученный BERT, который вам необходимо уменьшить без потерь в качестве. Задание разделено на две части. Первая часть состоит из набора методов по уменьшению модели, которые нужно реализовать по инструкции. Вторая часть – это творческое соревнование, в котором вы можете пользоваться любыми методами, кроме ансамблирования и использования дополнительных данных. Дополнительное условие соревнования: размер вашей модели __не может превышать 20M параметров__.

__!!ВАЖНО!!__ Вам придется проводить довольно много экспериментов, поэтому мы рекомендуем не писать весь код в тетрадке, а завести разные файлы для отдельных логических блоков и скомпоновать все в виде проекта. Это позволит вашему ноутбуку не разрастаться и сильно облегчит задачу и вам, и проверяющим.


### О датасете

В CoNLL-2003 для именования сущностей используется маркировка **BIO** (Beggining, Inside, Outside), в которой метки означают следующее:

- *B-{метка}* – начало сущности *{метка}*
- *I-{метка}* – продолжнение сущности *{метка}*
- *O* – не сущность

Существуют так же и другие способы маркировки, например, BILUO. Почитать о них можно [тут](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)) и [тут](https://www.youtube.com/watch?v=dQw4w9WgXcQ).

Всего в датасете есть 9 разных меток.
- O – слову не соответствует ни одна сущность.
- B-PER/I-PER – слово или набор слов соответстует определенному _человеку_.
- B-ORG/I-ORG – слово или набор слов соответстует определенной _организации_.
- B-LOC/I-LOC – слово или набор слов соответстует определенной _локации_.
- B-MISC/I-MISC – слово или набор слов соответстует сущности, которая не относится ни к одной из предыдущих. Например, национальность, произведение искусства, мероприятие и т.д.

Приступим!

In [194]:
#!g1.1
!!pip install transformers

['Defaulting to user installation because normal site-packages is not writeable',
 'Requirement already satisfied: transformers in /home/jupyter/.local/lib/python3.10/site-packages (4.36.2)',
 'Requirement already satisfied: filelock in /usr/local/lib/python3.10/dist-packages (from transformers) (3.12.2)',
 'Requirement already satisfied: huggingface-hub<1.0,>=0.19.3 in /home/jupyter/.local/lib/python3.10/site-packages (from transformers) (0.19.4)',
 'Requirement already satisfied: numpy>=1.17 in /usr/local/lib/python3.10/dist-packages (from transformers) (1.22.4)',
 'Requirement already satisfied: packaging>=20.0 in /kernel/lib/python3.10/site-packages (from transformers) (23.2)',
 'Requirement already satisfied: pyyaml>=5.1 in /usr/local/lib/python3.10/dist-packages (from transformers) (6.0.1)',
 'Requirement already satisfied: regex!=2019.12.17 in /usr/local/lib/python3.10/dist-packages (from transformers) (2022.10.31)',
 'Requirement already satisfied: requests in /usr/local/lib/py

In [195]:
#!g1.1
%load_ext autoreload
%autoreload 2

In [196]:
#!g1.1
import os
import numpy as np
import torch
from torch import nn
from transformers import AutoModelForTokenClassification, AutoTokenizer, DataCollatorForTokenClassification
from torch.utils.data import DataLoader
from typing import Dict, List

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
device

2023-12-19 11:51:02.556543: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


device(type='cuda')

__Задание 1 (0.5 балла)__ Допишите функцию `read_conll2003` для чтения датасета. Внутри она должна проитерироваться по всем строкам файла и для каждого примера составить словарь с полями `words` и `tags` (слова и тэги текста соответственно). На выход функция возвращает список полученных словарей. Тексты в файле разделяются переносом строки `\n`, а слова и тэги – проблелом. Пример:
```
! head -n 15 CoNLL2003/train.txt

EU B-ORG
rejects O
German B-MISC
call O
to O
boycott O
British B-MISC
lamb O
. O

Peter B-PER
Blackburn I-PER

BRUSSELS B-LOC
1996-08-22 O
```

In [197]:
#!g1.1
def read_conll2003(path: str) -> List[Dict[str, str]]:
    """
    Read data in CoNNL like format.
    """

    cur_text = {"words": [], "tags": []}
    dataset = []
    with open(path, 'r') as f:
        for line in f:
            if (line == '\n'):
                dataset.append(cur_text)
                cur_text = {"words": [], "tags": []}
            else:
                word, tag = line.replace('\n', '').split(' ')
                cur_text["words"].append(word)
                cur_text["tags"].append(tag)

    return dataset

Прочитаем тренировочный и валидационный датасеты.

In [198]:
#!g1.1
train_dataset = read_conll2003("CoNLL2003/train.txt")
valid_dataset = read_conll2003("CoNLL2003/valid.txt")

tags = ['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O']

In [199]:
#!g1.1
sample = train_dataset[0]

assert sample['words'] == ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
assert sample['tags'] == ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

for w, t in zip(sample['words'], sample['tags']):
    print(f'{w}\t{t}')

EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
lamb	O
.	O


На протяжении всего домашнего задания мы будем использовать _cased_ версию BERT, то есть токенизатор будет учитывать регистр слов. Для задачи NER регистр важен, так как имена и названия организаций или предметов искусства часто пишутся с большой буквы, и будет глупо прятать от модели такую информацию.

In [200]:
#!g1.1
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Заметьте, что при токенизации слова могут разделиться на несколько токенов (как слово `lamb` из примера ниже), из-за чего появится несоответствие между числом токенов и тэгов. Это несоответствие нам придется устранить вручную.

In [201]:
#!g1.1
inputs = tokenizer(sample['words'], is_split_into_words=True)
print('Слова: ', sample['words'])
print('Токены:', inputs.tokens())

Слова:  ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
Токены: ['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']


К счастью, из выхода токенизатора можно достать список с номерами слов, к которым относится каждый токен. Если номер встретился несколько раз подряд, то слово разделилось. Специальные символы не принадлежат никакому слову, поэтому их номер – `None`.

In [202]:
#!g1.1
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

__Задание 2 (0.5 балла)__ Допишите метод `get_inputs_and_aligned_labels` класса `Dataset`. Он принимает в себя объект из прочитанного выше датасета, токенизирует слова и выравнивает тэги. Выравнивание происходит следующим образом: если токен пренадлежит тому же слову, что и предыдущий токен, и его тэг начинается на `B`, то надо поменять `B` на `I`, потому что это уже продолжение сущности; в любом другом случае тэг токена остается таким же, какой был у соответствующего ему слова.

Метод позвращает словарь с полями `input_ids` – результат токенизации, `labels` – индексы тэгов для каждого токена из маппинга `tag2id`, для специальных символов в качестве лейбла укажите -100, так как это значение по умолчанию, которое игнорируется при подсчете кросс-энтропии в классе `CrossEntropyLoss`.

In [203]:
#!g1.1
class Dataset:
    def __init__(self, raw_dataset: List[Dict[str, str]], tag2id: Dict[str, int]):
        """
        :params:
        raw_dataset: output of read_conll2003 function
        tag2id: mapping from tag name to its id
        """
        self.dataset = raw_dataset
        self.tag2id = tag2id

    def _find_next(self, token, words, i, j):
        for c_i in range(i, len(words)):
            while (j < len(words[c_i])):
                if words[c_i][j:].startswith(token):
                    i = c_i
                    return i, j
                j += 1
            j = 0
        return i, j

    def get_inputs_and_aligned_labels(self, sample):
        """
        Aligns tags with tokens and returns dict with token ids and tag ids.
        """
        tokenized = tokenizer(sample['words'], is_split_into_words=True)
        tags = sample['tags']

        special_tokens = ['[CLS]', '[SEP]', '[PAD]']
        labels = []
        i = -1
        j = 0
        for token in tokenized['input_ids']:
            source_token = tokenizer.convert_ids_to_tokens(token)
            if source_token in special_tokens:
                labels.append(-100)
            else:
                if source_token.startswith('##'):
                    source_token = source_token[2:]
                    i, j = self._find_next(source_token, sample['words'], i, j)
                    new_tag = 'I' + tags[i][1:] if tags[i].startswith('B') else tags[i]
                    labels.append(self.tag2id[new_tag])
                else:
                    i, j = self._find_next(source_token, sample['words'], i, j)
                    labels.append(self.tag2id[tags[i]])

        assert len(tokenized['input_ids']) == len(labels), "ids and labels must be the same length"

        return {
            'input_ids': tokenized['input_ids'],
            'labels': labels
        }

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        return self.get_inputs_and_aligned_labels(sample)

    def __len__(self):
        return len(self.dataset)

In [204]:
#!g1.1
tag2id = {tag: i for i, tag in enumerate(tags)}
id2tag = {i: tag for tag, i in tag2id.items()}

train_dataset = Dataset(train_dataset, tag2id)
valid_dataset = Dataset(valid_dataset, tag2id)

In [205]:
#!g1.1
sample = train_dataset[0]

input_ids, labels = sample['input_ids'], sample['labels']
print(labels)
assert input_ids == [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102]
assert labels == [-100, 2, 8, 1, 8, 8, 8, 1, 8, 8, 8, -100]

for idx, token, label in zip(input_ids, tokenizer.convert_ids_to_tokens(input_ids), labels):
    tag = id2tag[label] if label != -100 else ''
    print(f'{idx}\t{token}\t{label}\t{tag}')

[-100, 2, 8, 1, 8, 8, 8, 1, 8, 8, 8, -100]
101	[CLS]	-100	
7270	EU	2	B-ORG
22961	rejects	8	O
1528	German	1	B-MISC
1840	call	8	O
1106	to	8	O
21423	boycott	8	O
1418	British	1	B-MISC
2495	la	8	O
12913	##mb	8	O
119	.	8	O
102	[SEP]	-100	


На данный момент наш датасет возвращает по индексу списки токенов и меток, но при формировании батча нам надо их дополнить паддингами. Для этого существует Collator – класс, который вызывается при формировании батча. Он принимает набор произвольных объектов из датасета и делает из них тензоры согласно инструкциям. Для задачи классификации последовательности имеется специальный `DataCollatorForTokenClassification`, который добавляет паддинги к токенам и меткам, что нам собственно и нужно.

In [206]:
#!g1.1
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [207]:
#!g1.1
batch = data_collator([train_dataset[i] for i in range(2)])
print('Поля:\n', batch.keys())
print('\nИндексы токенов:\n', batch['input_ids'])
print('\nИндексы меток:\n', batch['labels'])

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Поля:
 dict_keys(['input_ids', 'attention_mask', 'labels'])

Индексы токенов:
 tensor([[  101,  7270, 22961,  1528,  1840,  1106, 21423,  1418,  2495, 12913,
           119,   102],
        [  101,  1943, 14428,   102,     0,     0,     0,     0,     0,     0,
             0,     0]])

Индексы меток:
 tensor([[-100,    2,    8,    1,    8,    8,    8,    1,    8,    8,    8, -100],
        [-100,    3,    7, -100, -100, -100, -100, -100, -100, -100, -100, -100]])


Теперь мы готовы обернуть всю нашу красоту в `DataLoader`, по которому будем итерироваться при обучении.

In [208]:
#!g1.1
batch_size = 64

train_loader = DataLoader(
    train_dataset,
    collate_fn=data_collator,
    batch_size=batch_size,
    pin_memory=True,
    shuffle=True
)

valid_loader = DataLoader(
    valid_dataset,
    collate_fn=data_collator,
    batch_size=batch_size,
    pin_memory=True,
    shuffle=False
)

### Метрика

Для оценки качества NER чаще всего используется F1-мера. Разделяют два метода подсчета метрики:
1) Token-level: считается правильность предсказания отденьной метки для каждого токена.
2) Entity-level: считается правильность предсказания метки для всей сущности целиком независимо от того, сколько слов или токенов в нее входит.

Обычно предпочтение отдается второму способу, так как иначе, во-первых, качество зависит от токенизации, а во-вторых, если сущность состоит из нескольких слов и модель выставляет словам разные метки, то становится непонятно, к какому именно классу относить данную сущность. Для практики такой результат настолько же плох, насколько полное неугадывание класса, поэтому странно давать за это баллы.

Заметьте, предсказание `[I-PER', 'I-PER]` при верном `[B-PER', 'I-PER]` считается корректным, так как из него можно однозначно восстановить ответ, догадавшись, что не первом месте должно стоять `B-`. В то же время при верном `[B-PER', 'B-PER]` такое предсказание корректным не будет.

Для подсчета метрики будем использовать уже готовое [решение](https://huggingface.co/spaces/evaluate-metric/seqeval) из библиотеки `seqeval` (семейство `huggingface`). 

In [209]:
#!g1.1
!!pip install seqeval
!!pip install transformers[torch]

['Defaulting to user installation because normal site-packages is not writeable',
 'Requirement already satisfied: transformers[torch] in /home/jupyter/.local/lib/python3.10/site-packages (4.36.2)',
 'Requirement already satisfied: filelock in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (3.12.2)',
 'Requirement already satisfied: huggingface-hub<1.0,>=0.19.3 in /home/jupyter/.local/lib/python3.10/site-packages (from transformers[torch]) (0.19.4)',
 'Requirement already satisfied: numpy>=1.17 in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (1.22.4)',
 'Requirement already satisfied: packaging>=20.0 in /kernel/lib/python3.10/site-packages (from transformers[torch]) (23.2)',
 'Requirement already satisfied: pyyaml>=5.1 in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (6.0.1)',
 'Requirement already satisfied: regex!=2019.12.17 in /usr/local/lib/python3.10/dist-packages (from transformers[torch]) (2022.10.31)',
 'Requirement

In [210]:
#!g1.1
from seqeval.metrics import f1_score, accuracy_score

In [211]:
#!g1.1
# here are 7 labels in total, we guessed correctly 4 of them.

predictions = [['O', 'I-PER', 'I-PER', 'O'], ['I-PER', 'I-PER', 'O']]
references = [['O', 'B-PER', 'B-PER', 'O'], ['B-PER', 'I-PER', 'O']]
acc = accuracy_score(predictions, references)
f1 = f1_score(references, predictions)
acc, f1

(0.5714285714285714, 0.4)

In [212]:
#!g1.1
def calc_f1(predictions: List[List[int]], labels: List[List[int]]):
    """
    :params:
    predictions: list of lists of predicted labels
    labels: list of lists of ground truth labels
    """
    text_labels = [[id2tag[l] for l in label if l != -100] for label in labels]
    text_predictions = []
    for i in range(len(text_labels)):
        # +1 because we skip the first ([CLS]) token
        sample_text_preds = [id2tag[predictions[i][j + 1]] for j in range(len(text_labels[i]))]
        text_predictions.append(sample_text_preds)

    return f1_score(text_labels, text_predictions)

### Модель

В качестве начальной модели мы будем использовать предобученный BERT, а если быть точнее `bert-base-cased` из библиотеки `huggingface`. Он содержит 107М параметров. В последующих заданиях мы будем реализовывать методы для уменьшения его размеров с минимальной потерей качества.

Для классификации последовательностей в `transformers` существует специальная обертка `AutoModelForTokenClassification`. Воспользуемся ею и обернем нашу модель.

In [217]:
#!g1.1
model = AutoModelForTokenClassification.from_pretrained('bert-base-cased', id2label=id2tag, label2id=tag2id).to(device)
print('Number of parameters:', sum([p.numel() for p in model.parameters()]))
teacher_model = model

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of parameters: 107726601


## Обучение всякого

**Задание 3 (1 балл)** Все методы уменьшения размерности основываются на том, что у нас есть некоторая обученная модель. Сейчас у нас есть предобученный BERT, но на задачу MLM, а не NER. Дообучите BERT на нашем датасете. Ориентировочно у вас должно получиться значение F1 не меньше 0.93 на валидационной выборке. Само обучение никак не должно занимать больше получаса.

In [218]:
#!g1.1
from transformers import Trainer, TrainingArguments
import numpy as np

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "f1 score": calc_f1(predictions, labels)
    }

def train_bert(model, data_collator, train_dataset, valid_dataset, args_dir="training_args", epochs=3.0):
    args = TrainingArguments(args_dir,
                        evaluation_strategy="epoch",
                        save_strategy="no",
                        num_train_epochs=epochs
                        )

    trainer = Trainer(model,
                      args=args,
                      data_collator=data_collator,
                      train_dataset=train_dataset,
                      eval_dataset=valid_dataset,
                      compute_metrics=compute_metrics)
    trainer.train()
    return model

In [219]:
#!g1.1
teacher_model = train_bert(model, data_collator, train_dataset, valid_dataset)

Epoch,Training Loss,Validation Loss,F1 score
1,0.094500,0.087247,0.909037
2,0.036800,0.078610,0.922407
3,0.018000,0.064889,0.935960


### Embedding factorization

Можно заметить, что на данный момент матрица эмбеддингов занимает $V \cdot H = 28996 \cdot 768 = 22.268.928$ параметров. Это целая пятая часть от всей модели! Давайте попробуем с этим что-то сделать. В вариации [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) предлагается факторизовать матрицу эмбеддингов в произведение двух небольших матриц. Таким образом, параметры эмбеддингов будут содержать $V \cdot E + E \cdot H$ элементов, что гораздо меньше, если $H \gg E$. Авторы выбирают $E = 128$, однако ничего не мешает вам взять значение меньше.

__Задание 4 (1 балл)__ Замените слой эмбеддингов на описанную факторизацию и дообучите полученную в предыдущем задании модель. Насколько вам удалось уменьшить число параметров? Если вы все сделали правильно, то F1-мера на валидации не должна опуститься ниже 0.9.

Мы настоятельно рекомендуем переиспользовать код для обучения из предыдущего задания и не создавать новую функцию.

In [ ]:
#!g1.1
from module import EmbedFactorized
model.bert.embeddings.word_embeddings = EmbedFactorized(28996, 768, E=128, padding_idx=0, embedding_matrix=model.bert.embeddings.word_embeddings.weight.data)
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): EmbedFactorized(
        (emb1): Embedding(28996, 128, padding_idx=0)
        (emb2): Embedding(128, 768, padding_idx=0)
      )
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=

In [ ]:
#!g1.1
print('Number of parameters:', sum([p.numel() for p in model.parameters()]))

Number of parameters: 89267465


In [ ]:
#!g1.1
train_bert(model, data_collator, train_dataset, valid_dataset, "embedding_factorization", epochs=5.0)

Epoch,Training Loss,Validation Loss,F1 score
1,0.191700,0.214531,0.781443
2,0.095400,0.150203,0.853233
3,0.060100,0.151515,0.854876
4,0.033100,0.165267,0.869291
5,0.018200,0.169136,0.882552


### Дистилляция знаний

Дистилляция знаний – это парадигма обучения, в которой знания модели-учителя дистиллируются в модель-ученика. Учеником может быть произвольная модель меньшего размера, решающая ту же задачу. При дистилляции используются два функционала ошибки:

1. Стандартная кросс-энтропия.
1. Функция, задающая расстояние между распределениями предсказаний учителя и ученика. Чаще всего используют кросс-энтропию или KL-дивергенцию.

При этом для того, чтобы распределение предсказаний учителя не было таким вырожденным используют softmax с температурой больше 1, например, 2 или 5.

<img src="https://intellabs.github.io/distiller/imgs/knowledge_distillation.png">

__Задание 5 (1 балл)__ Реализуйте метод дистилляции знаний, изображенный на картинке. Для подсчета ошибки между предсказаниями ученика и учителя используйте KL-дивергенцию (`nn.KLDivLoss(reduction="batchmean")`). В качестве учителя используйте дообученный BERT из задания 3. В качестве ученика вы можете взять произвольную необученную модель с размером около 40M параметров. Не забудьте про warmup!

In [222]:
#!g1.1
from trainer import DistilTrainer

def train_distil(student_model, teacher_model, data_collator, train_dataset, valid_dataset, args_dir="training_args", epochs=3.0):
    args = TrainingArguments(args_dir,
                        evaluation_strategy="epoch",
                        save_strategy="no",
                        num_train_epochs=epochs,
                        warmup_ratio=0.06,
                        weight_decay=1e-5,
                        lr_scheduler_type="linear"
                        )

    trainer = DistilTrainer(student_model,
                      teacher_model=teacher_model,
                      temperature=2.0,
                      lambda_param=0.5,
                      args=args,
                      data_collator=data_collator,
                      train_dataset=train_dataset,
                      eval_dataset=valid_dataset,
                      compute_metrics=compute_metrics)
    trainer.train()
    return student_model

In [225]:
#!g1.1
from transformers import DistilBertForTokenClassification, DistilBertConfig

configuration = DistilBertConfig(dim=384, num_labels=9)
student_model = DistilBertForTokenClassification(configuration).to(device)
print('Number of student parameters:', sum([p.numel() for p in student_model.parameters()]))

print('Number of teacher parameters:', sum([p.numel() for p in teacher_model.parameters()]))

distilled_model = train_distil(student_model, teacher_model, data_collator, train_dataset, valid_dataset, args_dir="distil_bert", epochs=100.0)

Number of student parameters: 29655177
Number of teacher parameters: 107726601


Epoch,Training Loss,Validation Loss,F1 score
1,54.093000,39.320770,0.124388
2,36.174500,27.779524,0.351824
3,24.214500,20.988920,0.472730
4,19.279900,18.165960,0.543808
5,15.475200,16.031973,0.583740
6,13.127300,15.051217,0.607172
7,11.038200,13.887386,0.669031
8,9.546200,13.180466,0.680142
9,7.977600,12.670936,0.703784
10,7.052700,12.587157,0.706190


## Соревнование (до 10 баллов)

Ваша задача – обучить модель с размером __не больше 20М параметров__ для задачи NER. При этом можно пользоваться предобученным `bert-base-cased`, но больше ничем. 

Соревнование будет проходить аналогично соревнованию из второго домашнего задания. Ваши посылки вы должны будете отправлять тг-боту [@nlp_hw4_bot](t.me/nlp_hw4_bot), а он будет считать значения F1 на публичном и приватном датасетах и записывать результат в [табличку](https://docs.google.com/spreadsheets/d/1rILRI16VxgztwlfqR2kPZ3MxlJkerz6iEr5Kx9yrOLA/edit#gid=0).

Для формирования посылки вам нужно будет создать папку на dropbox, положить в нее файл `model.py` с классом модели `Model` и веса `weights.pt`, а затем отправить боту ссылку на эту папку, доступную к чтению. Бот будет импортировать модель и загружать веса:
```
module = __import__('model', globals(), locals(), ['Model'], 0)
model = module.Model()
model.load_state_dict(torch.load('weights.pt', map_location=torch.device('cpu')))
```

При тестировании модель будет получать на вход `input_ids` и `attention_mask`, а на выход должна возвращать трехмерный тензор с вероятностями меток для каждого токена в батче. Класс `Model` должен содержать атрибут `id2label` совпадающий с тем, который задан в конфиге модели `model.config.id2label`. Это нужно для того, чтобы id тэгов мапились в нужные названия тэгов, так как они могут отличаться у разных решений.


__Обязательм условием__ участия в соревновании является отчет о проделанной работе в формате pdf, в котором вы должны описать опробованные методы с результатами. За отчет выставляется максимум до __2 баллов__ на усмотрение проверяющего. В случае отсутствия отчета баллы за соревнование __обнуляются__.

После дедлайна по домашке будут выложен _приватный_ лидерборд, по которому и будут выставляться баллы за соревнования. За место в лидерборде можно получить до __8__ баллов, но только при условии, если вы получили больше __0.8__ на _публичном_ лидерборде, в противном случае баллы выставляться не будут.
$$
\text{число баллов} = 8\frac{(N - r + 1)}{N},
$$
где $r$ – место в лидерборде, а $N$ - число участников со значением F1 на _публичном_ лидерборде не меньше __0.8__.


На сервере установлена версия библиотеки `transformers==4.34.0`.
В разных версиях может отличаться вид хранения весов, поэтому рекомендуем установить себе такую же версию, чтобы избежать ошибок при загрузке модели.

### Что стоит попробовать?

* В статье [ALBERT](https://arxiv.org/abs/1909.11942) помимо факторизации эмбеддингов предлагается использовать одни и те же параметры для нескольких слоев. Такой подход позволяет серьезно уменьшить число параметров.

* В задании 5 мы инициализировали ученика случайно, однако можно сделать лучше. При дистилляции знаний для downstream задачи из предобученного в unsupervised формате учителя (задача MLM) часто помогает сперва дистиллировать модель для задачи предобучения, а затем ее уже дообучать на downstream задачу с соответствующим учителем. Другими словами, лучше сначала дистиллировать предобученный BERT в ученика на MLM задаче, а затем использовать этого ученика в качестве начальной инициализации для второй дистилляции.

* При дистилляции мы выравниваем только предсказания моделей, однако можно выравнивать еще и скрытые слои. Например, приближать матрицы внимания и выходы каждого скрытого слоя. Подробнее об этом можно почитать [тут](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT).

* В данный момент мы используем все головы внимания, но ряд исследований показывает, что большинство из них можно выбросить без потери качества. В этой [статье](https://arxiv.org/pdf/1905.09418.pdf) предлагается следующий подход. Добавим гейт $g_i \in \{0, 1\}$ для каждой головы внимания.\
$$
\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_I^V)
$$
$$
\text{MultiHeadAttention}(Q, K, V) = \text{Concat}(g_i \cdot \text{head}_i) W^O
$$
---
__Теоретический блок для тех, кому интересно__   
Будем настраивать значения гейтов в процессе обучения. Мы хотим, чтобы как можно большая часть гейтов стала нулями, поэтому добавим в функционал модели $L_0$ регуляризацию на $g_i$. Проблема в том, что $L_0$ – недифференцируемая функция, поэтому нам надо релаксировать ее.
Будем считать, что каждый гейт задается распределением Бернулли, $g_i \sim \text{Bernoulli}(\alpha_i)$, где $\alpha_i$ – настраиваемый параметр. Осталось понять, как его настраивать. Если мы будем напрямую семплировать из распределения Бернулли, то мы потеряем связь между $\alpha_i$ и семплом, поэтому мы не сможем прокинуть градиенты (такая же проблема возникает в VAE, там используют reparametrization trick). Существует хороший способ семплирования дискретных случайных величин с сохранением связи – __Gumbel-Max trick__. Пусть вероятность каждого значения случайной величины пропорциональна $\beta_k \in (0, \infty)$ и \
$$
x = \text{argmax}_k \{\log \beta_k - \log(-\log(\text{Uniform}(0, 1))) \},
$$
Тогда $P(x = k) = \frac{\beta_k}{\sum_k \beta_k}$. Для того, чтобы избавиться от недифференцируемого аргмакса можно релаксировать его, заменив на softmax с температурой меньше 1, так мы получим [Concrete distribution](https://arxiv.org/pdf/1611.00712.pdf). Теперь мы сможем в процессе обучения семплировать значения гейтов и обновлять $\alpha_i$ градиентным спуском. Почти победа, осталось понять, при каких значениях $\alpha_i$ после обучения мы будем считать, что гейт закрыт и голову можно выбросить. В [статье](https://arxiv.org/pdf/1712.01312.pdf) про Hard Concrete распределение, предложившей $L_0$ регуляризацию, предлагается немного растянуть распределение вероятности открытия гейта с $[0, 1]$ до $[\gamma, \zeta]$ (например, $[-0.1, 1.1]$), а затем обрезать его обратно до $[0, 1]$. Таким образом, все значения, которые были меньше 0, превратятся в 0. Теперь мы будем считать гейт закрытым, если мы получили на выходе 0.
$$g_i = \text{clip}\big(\text{Sigmoid}(\log \alpha_i)(\zeta - \gamma) + \gamma, 0, 1\big)$$
---
Получаем следующий алгоритм подбора значений для гейтов.
1. Заводим параметр $\log \alpha_i$ для каждой головы каждого слоя.
2. Добавляем к функционалу ошибки слагаемое регуляризации с небольшим коэффициентом $\lambda$ (например, $0.02$)
$$
\mathcal{L}_C = \sum_{i=1}^h (1 - P(g_i = 1 | \alpha_i)) = \sum_{i=1}^h \text{Sigmoid}\Big(\log \alpha_i - \tau \log \frac{-\gamma}{\zeta}\Big),
$$
где $\gamma$ < 0 и $\zeta$ > 1, $\tau$ – гиперпараметры (можно взять $-0.1$, $1.1$ и $0.33$ соответственно)
3. При каждом вызове модели значения гейтов семплируются из Hard Concrete распределения.
\begin{align}
u &= \text{Uniform}(0, 1) \\
z &= \text{Sigmoid}\big((\log u - \log(1 - u) + \log \alpha_i) \,/\, \tau\big) \\
g_i &= \text{clip}\big( z (\zeta - \gamma) + \gamma, 0, 1 \big)
\end{align}
4. После обучения в идеале выбрасываются головы, для которых $\text{clip}\big(\text{Sigmoid}(\log \alpha_i)(\zeta - \gamma) + \gamma, 0, 1\big)$ равняется 0. Если таких нет или очень мало, то можно выбросить те, которые близки к нулю, а затем дообучить модель без этих голов.


[Тут](https://arxiv.org/pdf/2110.03252.pdf) можно почитать про дополнительные хаки для этого метода.\
\
P. S. Заводится тяжело, но заводится. Если гейты не начинают зануляться, то, возможно, вы недостаточно долго учите.

---

Помимо всего, что написано выше, на просторах интернета можно найти кучу других способов уменьшения модели, не стестяйтесь их искать. При проведении экспериментов старайтесь делать одно изменение за раз и не бросайтесь реализовывать сложный метод, если у вас нет достаточных оснований полагать, что он даст значительный буст.

Удачи!

In [ ]:
#!g1.1
